<a href="https://colab.research.google.com/github/HYLee810/projectmanager/blob/main/Bert_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets accelerate evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 104.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.5 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base") # klue/bert-base라는 모델을 사용하는 것


from transformers import AutoModelForSequenceClassification
seq_model = AutoModelForSequenceClassification.from_pretrained(
    "klue/bert-base", num_labels=7,
    id2label={
        0: 'IT과학', 1: '경제',  2: '사회',  3: '생활문화', 4: '세계',
        5: '스포츠',  6: '정치'}) # klue/bert-base라는 모델이 구분하는 레이블
seq_model.to('cuda:0')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [3]:
from datasets import load_dataset
ynat_train = load_dataset('klue', 'ynat', split='train')
ynat_val = load_dataset('klue', 'ynat', split='validation')

Generating train split:   0%|          | 0/45678 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9107 [00:00<?, ? examples/s]

In [4]:
ynat_train[0] # huggingface - 연합뉴스로 만들어 놓은 예제

{'guid': 'ynat-v1_train_00000',
 'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영',
 'label': 3,
 'url': 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=227&oid=001&aid=0008508947',
 'date': '2016.06.30. 오전 10:36'}

In [7]:
def convert_example (example):
    result = tokenizer(example['title'])
    result['label'] = [example['label']]
    return result

ynat_train_dataset = ynat_train.map(convert_example)
ynat_val_dataset = ynat_val.map(convert_example)

Map:   0%|          | 0/45678 [00:00<?, ? examples/s]

Map:   0%|          | 0/9107 [00:00<?, ? examples/s]

In [8]:
from transformers import TrainingArguments , Trainer
training_args= TrainingArguments(
  output_dir="test_trainer",
  num_train_epochs=1,
  evaluation_strategy ="epoch")

from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=seq_model,
    args=training_args,
    train_dataset=ynat_train_dataset,
    eval_dataset=ynat_val_dataset,
    data_collator=data_collator
)

trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.342400,0.404953


TrainOutput(global_step=5710, training_loss=0.4554967997161812, metrics={'train_runtime': 567.7158, 'train_samples_per_second': 80.459, 'train_steps_per_second': 10.058, 'total_flos': 463740691731900.0, 'train_loss': 0.4554967997161812, 'epoch': 1.0})

In [9]:
from transformers import pipeline
text_cls = pipeline('text-classification', model=seq_model,
              tokenizer=tokenizer, device='cuda:0')

In [10]:
text_cls('2회 연속 은메달 우상혁 "아쉽지만 파리올림픽 있어."')

[{'label': '스포츠', 'score': 0.9944274425506592}]